In [1]:
from skimage import feature
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from imutils import paths
import numpy as np
import argparse
import cv2
import os
import random


In [2]:
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints,
                                           self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
                                 bins=np.arange(0, self.numPoints + 3),
                                 range=(0, self.numPoints + 2))

        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist
    

In [3]:
def make_dataset():
    all_img = []
    for i in os.listdir(root_path):
        if (i == '.DS_Store'):
            continue
        for j in os.listdir(os.path.join(root_path, i)):
            if (j == '.DS_Store'):
                continue
            all_img.append(os.path.join(root_path, i, j))
    random.shuffle(all_img)
    return all_img[30:], all_img[:30]


In [4]:
root_path = 'images/dataset'


train_path, test_path = make_dataset()
desc = LocalBinaryPatterns(24, 8)
data = []
labels = []


for imagePath in train_path:
    print(imagePath)
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = desc.describe(gray)
    labels.append(imagePath.split('/')[-2])
    data.append(hist)

images/dataset/5/22.tif
images/dataset/7/46.tif
images/dataset/2/45.tif
images/dataset/5/21.tif
images/dataset/9/44.tif
images/dataset/2/15.tif
images/dataset/3/42.tif
images/dataset/1/12.tif
images/dataset/5/13.tif
images/dataset/4/55.tif
images/dataset/0/65.tif
images/dataset/7/35.tif
images/dataset/4/25.tif
images/dataset/3/31.tif
images/dataset/1/21.tif
images/dataset/0/16.tif
images/dataset/5/32.tif
images/dataset/7/66.tif
images/dataset/9/46.tif
images/dataset/1/41.tif
images/dataset/1/61.tif
images/dataset/4/52.tif
images/dataset/5/41.tif
images/dataset/2/16.tif
images/dataset/0/43.tif
images/dataset/3/43.tif
images/dataset/4/36.tif
images/dataset/7/32.tif
images/dataset/1/52.tif
images/dataset/9/63.tif
images/dataset/7/22.tif
images/dataset/2/43.tif
images/dataset/1/32.tif
images/dataset/8/32.tif
images/dataset/1/34.tif
images/dataset/7/43.tif
images/dataset/5/15.tif
images/dataset/8/14.tif
images/dataset/1/36.tif
images/dataset/7/31.tif
images/dataset/2/52.tif
images/dataset/6

In [5]:
# model = svm.SVC(decision_function_shape='ovo')
from sklearn import tree
from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score

model = tree.DecisionTreeClassifier()
# model = KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree')
model.fit(data, labels)
# print(model.classes_)
# c = 0
# cc = 0
y_prediction = []
y_true = []
for imagePath in test_path:
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = desc.describe(gray)
    y_prediction.append(model.predict(hist.reshape(1, -1)))
    y_true.append(imagePath.split('/')[-2])
#     if (model.predict(hist.reshape(1, -1)) == imagePath.split('/')[-2]):
#         c +=1
#     else:
#         cc+=1
print(confusion_matrix(y_true, y_prediction))
print(precision_score(y_true, y_prediction,average=None))
print(recall_score(y_true, y_prediction, average=None))
print(accuracy_score(y_true, y_prediction))

    
# print(c/(cc+c))

[[1 0 0 0 0 1 0 0 0]
 [0 3 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 2]
 [0 1 0 2 0 0 0 0 0]
 [0 0 0 1 2 0 0 0 0]
 [0 1 0 1 0 2 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 7 0]
 [0 1 0 0 0 0 0 0 1]]
[1.         0.5        1.         0.5        1.         0.66666667
 1.         1.         0.33333333]
[0.5        1.         0.6        0.66666667 0.66666667 0.5
 1.         1.         0.5       ]
0.7333333333333333
